In [1]:
import pandas as pd
from torch import nn
import torch
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [2]:
test = pd.read_csv(r'C:\Users\vvagh\OneDrive - Indian Institute of Science Education and Research Bhopal\Documents\kaggle\s4e7\test.csv\test.csv')
train = pd.read_csv(r'C:\Users\vvagh\OneDrive - Indian Institute of Science Education and Research Bhopal\Documents\kaggle\s4e7\train.csv\train.csv')

In [3]:
train

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,11504793,Male,48,1,6.0,0,1-2 Year,Yes,27412.0,26.0,218,0
11504794,11504794,Female,26,1,36.0,0,< 1 Year,Yes,29509.0,152.0,115,1
11504795,11504795,Female,29,1,32.0,1,< 1 Year,No,2630.0,152.0,189,0
11504796,11504796,Female,51,1,28.0,0,1-2 Year,Yes,48443.0,26.0,274,1


# data cleaning nd preprocessing

policy_sales cahnnel is not imp

In [4]:
train['Vehicle_Age'].unique()

array(['1-2 Year', '> 2 Years', '< 1 Year'], dtype=object)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 1.0+ GB


In [6]:
train['Gender']=train['Gender'].astype(str)
train['Vehicle_Age']=train['Vehicle_Age'].astype(str)
train['Vehicle_Damage']=train['Vehicle_Damage'].astype(str)

In [7]:
le=LabelEncoder()
train['Gender'] = le.fit_transform(train['Gender'])
train['Vehicle_Damage'] = le.fit_transform(train['Vehicle_Damage'])

In [8]:
yera_mapping = {'1-2 Year': 1.5, '> 2 Years': 2.1,'< 1 Year':0.9}

train['Vehicle_Age'] = train['Vehicle_Age'].map(yera_mapping)
test['Vehicle_Age'] = test['Vehicle_Age'].map(yera_mapping)


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                int32  
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           float64
 7   Vehicle_Damage        int32  
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(4), int32(2), int64(6)
memory usage: 965.5 MB


In [10]:
train=train[['Gender','Age','Driving_License','Region_Code','Previously_Insured','Vehicle_Age','Vehicle_Damage','Annual_Premium','Response']]

In [11]:
scaler = MinMaxScaler()
train[['Region_Code','Age','Annual_Premium']] = scaler.fit_transform(train[['Region_Code','Age','Annual_Premium']])

C:\Users\vvagh\AppData\Local\Temp\ipykernel_4420\950311951.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[['Region_Code','Age','Annual_Premium']] = scaler.fit_transform(train[['Region_Code','Age','Annual_Premium']])


In [12]:
train

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Response
0,1,0.015385,1,0.673077,0,1.5,1,0.116218,0
1,1,0.353846,1,0.538462,0,2.1,1,0.104702,1
2,0,0.076923,1,0.269231,1,0.9,0,0.065880,0
3,0,0.230769,1,0.019231,0,1.5,1,0.000000,0
4,0,0.246154,1,0.288462,1,1.5,0,0.054547,0
...,...,...,...,...,...,...,...,...,...
11504793,1,0.430769,1,0.115385,0,1.5,1,0.046103,0
11504794,0,0.092308,1,0.692308,0,0.9,1,0.050004,1
11504795,0,0.138462,1,0.615385,1,0.9,0,0.000000,0
11504796,0,0.476923,1,0.538462,0,1.5,1,0.085228,1


In [13]:
#making the data ready for the model
labels=train['Response']
train=train.drop(['Response'],axis=1)

# Convert DataFrames to NumPy arrays first
train_features = train.values
train_labels = labels.values

# Convert NumPy arrays to PyTorch tensors
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)  # Use dtype=torch.long if labels are integers


In [14]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create the dataset
dataset = CustomDataset(train_features_tensor, train_labels_tensor)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [15]:
class logistic_regression(nn.Module):
    def __init__(self):
        super(logistic_regression, self).__init__()
        self.linear = nn.Linear(8, 10)
        self.hidden=nn.Linear(10,10)
        self.relu=nn.ReLU()
        self.output=nn.Linear(10,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        x=self.linear(x)
        x=self.relu(x)
        x=self.hidden(x)
        x=self.relu(x)
        x=self.output(x)
        x=self.sigmoid(x)
        return (x)

In [16]:
model=logistic_regression()
loss=nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
epochs=5
for epoch in range(epochs):
    epoch_loss = 0
    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", unit='batch') as pbar:
        for batch in pbar:
            features, labels = batch
            optimizer.zero_grad()
            predictions = model(features)
            predictions = predictions.squeeze(1)
            loss_value = loss (predictions, labels)
            loss_value.backward()
            optimizer.step()
            
            # Accumulate the loss for the epoch
            epoch_loss += loss_value.item()
    
    avg_loss = epoch_loss / len(dataloader)
    # Print the average loss for the epoch below the progress bar
    print(f'Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}')


Epoch 1/5: 100%|██████████| 359525/359525 [18:09<00:00, 329.98batch/s]   


Epoch 1/5, Average Loss: 0.2691


Epoch 2/5: 100%|██████████| 359525/359525 [07:44<00:00, 773.23batch/s]


Epoch 2/5, Average Loss: 0.2688


Epoch 3/5: 100%|██████████| 359525/359525 [10:11<00:00, 588.14batch/s] 


Epoch 3/5, Average Loss: 0.2688


Epoch 4/5: 100%|██████████| 359525/359525 [05:57<00:00, 1005.79batch/s]


Epoch 4/5, Average Loss: 0.2688


Epoch 5/5: 100%|██████████| 359525/359525 [05:50<00:00, 1026.25batch/s]

Epoch 5/5, Average Loss: 0.2688
